In [1]:
import numpy as np
import pandas as pd
import json
from datetime import datetime
from datetime import timedelta
import gc
from math import radians, cos, sin, asin, sqrt
import pickle

In [2]:
pd.set_option("display.max_columns", 800)

In [3]:
# add recent against team's win percentage.
teams = json.loads(open("/Users/chienan/Pycon/github/SportLottery/nbastat/teamId_mpt").read())

In [ ]:
df = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/TEAMGAMELOG_2012to2019.pkl").sort_values("Game_ID").reset_index(drop=True)

In [ ]:
df.WL = df.WL.replace({"W":1,"L":0})

In [ ]:
lastNgames = 10 

In [ ]:
# Aggregate Data
df_ = pd.DataFrame()
for key, team_name in zip(teams.keys(),teams.values()):
    df_i = df[df.Team_ID==int(key)].reset_index(drop=True)
    df_team = pd.DataFrame()
    for times in range(1,lastNgames+1):
        mean_lst = []
        sum_lst = []
           
        for time in range(1, times+1):
            mean_lst.append(df_i.shift(time).loc[:,"FGM":"PTS"].values)
            sum_lst.append(df_i.shift(time).loc[:,"WL":"WL"].values)
        df_team = pd.concat([df_team,
                             pd.DataFrame(np.mean(np.array(mean_lst),axis=0),
                                          columns=["LAST_"+str(times)+"_GAME_"+col for col in df_i.loc[:,"FGM":"PTS"].columns]),
                             pd.DataFrame(np.sum(np.array(sum_lst),axis=0),
                                          columns=["LAST_"+str(times)+"_GAME_WL"])],axis=1)
        
    df_team = pd.concat([df_i.loc[:,"Team_ID":"MATCHUP"],df_team],axis=1)
    df_ = pd.concat([df_, df_team],axis=0).reset_index(drop=True)
    print("finish %s"%team_name)

In [ ]:
df_.to_pickle("/Users/chienan/Pycon/github/SportLottery/input/TEAMGAMELOG_LAST_10_GAEMS_AGG_2012to2019.pkl")

# Team Data Combine

In [ ]:
teams = json.loads(open("/Users/chienan/Pycon/github/SportLottery/nbastat/teamId_mpt").read())

In [ ]:
# Aggregate Data
df_ = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/TEAMGAMELOG_LAST_10_GAEMS_AGG_2012to2019.pkl")

In [ ]:
len(df_[["Team_ID","Game_ID"]].drop_duplicates())

In [ ]:
plus_minus = pd.concat([pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_AWAY_2012to2019.pkl"),
                        pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_HOME_2012to2019.pkl")],
                       axis=0).reset_index(drop=True)

In [ ]:
plus_minus = plus_minus[['GAME_ID', 'TEAM_ID', 'GAME_DATE', 'PLUS_MINUS']]

In [ ]:
plus_minus.head()

In [ ]:
lastNgames = 10 

In [ ]:
# Aggregate Data
plus_minus_ = pd.DataFrame()
for key, team_name in zip(teams.keys(),teams.values()):
    df_i = plus_minus[plus_minus.TEAM_ID==int(key)].reset_index(drop=True)
    df_team = pd.DataFrame()
    for times in range(1,lastNgames+1):
        mean_lst = []

        
        for time in range(1, times+1):
            mean_lst.append(df_i.shift(time).loc[:,"PLUS_MINUS":"PLUS_MINUS"].values)
            
        df_team = pd.concat([df_team,pd.DataFrame(np.mean(np.array(mean_lst),axis=0),
                                                  columns=["LAST_"+str(times)+"_GAME_PLUS_MINUS"])],axis=1)
    df_team = pd.concat([df_i.loc[:,"GAME_ID":"TEAM_ID"],df_team],axis=1)
    plus_minus_ = pd.concat([plus_minus_, df_team],axis=0).reset_index(drop=True)
    print("finish %s"%team_name)

In [ ]:
# Home Team Data
nba_home_df = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_HOME_2012to2019.pkl")
nba_home_df = nba_home_df[['TEAM_ID','GAME_ID','PLUS_MINUS']]

# Away Team Data
nba_away_df = pd.read_pickle('/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_AWAY_2012to2019.pkl')
nba_away_df = nba_away_df[['TEAM_ID','GAME_ID']]

# Join Two Table
nba_df = nba_home_df.merge(nba_away_df, on='GAME_ID', suffixes=['_HOME','_AWAY'],how='inner')
nba_df = nba_df[['GAME_ID','TEAM_ID_HOME','TEAM_ID_AWAY','PLUS_MINUS']]
nba_df.head()

In [ ]:
nba_df = nba_df.merge(df_,left_on=['TEAM_ID_HOME','GAME_ID'],right_on=['Team_ID','Game_ID'],how='inner')
nba_df = nba_df.merge(df_,left_on=['TEAM_ID_AWAY','GAME_ID'],right_on=['Team_ID','Game_ID'],
                      suffixes=['_H','_A'],how='inner')
nba_df['GAME_DATE']=nba_df['GAME_DATE_H']

In [ ]:
for col in [col for col in nba_df.columns if ("Team_" in col)|("Game_ID" in col)|("MATCHUP" in col)|("GAME_DATE_" in col)]:
    del nba_df[col]

In [ ]:
gc.collect()

In [ ]:
nba_df.GAME_DATE = nba_df.GAME_DATE.apply(lambda dt : datetime.strptime(dt, "%b %d, %Y"))

In [ ]:
nba_df = nba_df.merge(plus_minus_,left_on=['TEAM_ID_HOME','GAME_ID'],right_on=['TEAM_ID','GAME_ID'],how='inner')
nba_df = nba_df.merge(plus_minus_,left_on=['TEAM_ID_AWAY','GAME_ID'],right_on=['TEAM_ID','GAME_ID'],
                      suffixes=['_H','_A'],how='inner')

In [ ]:
nba_df.tail()

# Player Data Combine

In [ ]:
playerBoxScoreAdv = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/BOXSCOREADVANCE_PLAYER_2012to2019.pkl")

In [ ]:
playerBoxScoreAdv = playerBoxScoreAdv[~playerBoxScoreAdv.MIN.isnull()]
playerBoxScoreAdv['MIN'] = playerBoxScoreAdv.MIN.apply(lambda x : float(x.split(':')[0]) + float(x.split(':')[1])/60.0)

In [ ]:
playerBoxScoreAdv = playerBoxScoreAdv.loc[:,['GAME_ID','PLAYER_ID','TEAM_ID','MIN', 'OFF_RATING','DEF_RATING', 
                                             'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO','OREB_PCT', 'DREB_PCT',
                                             'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT','USG_PCT', 'PACE', 'PIE']]

In [ ]:
playerBoxScoreAdv = playerBoxScoreAdv.fillna(0)

In [ ]:
playerBoxScoreAdv["TEAM_ID"] = playerBoxScoreAdv["TEAM_ID"].astype(str)
playerBoxScoreAdv["PLAYER_ID"] = playerBoxScoreAdv["PLAYER_ID"].astype(str)

In [ ]:
playerBoxScoreAdv = playerBoxScoreAdv.sort_index()

In [ ]:
player_id = playerBoxScoreAdv.PLAYER_ID.unique()

In [ ]:
playerBoxScoreAdv.index = [playerBoxScoreAdv.PLAYER_ID.values,playerBoxScoreAdv.GAME_ID.values]
playerBoxScoreAdv = playerBoxScoreAdv.sort_index()

In [ ]:
player_id

In [ ]:
# Aggregate Data
df_ = pd.DataFrame()
for player in player_id:
    player_i = playerBoxScoreAdv.loc[player]
    df_player = pd.DataFrame()
    
    for times in range(1,lastNgames+1):
        mean_lst = []
        for time in range(1, times+1):
            mean_lst.append(player_i.shift(time).loc[:,"MIN":].values)
        df_player = pd.concat([df_player,
                               pd.DataFrame(np.mean(np.array(mean_lst),axis=0),
                                            columns=["LAST_"+str(times)+"_GAME_PLAYER_"+col for col in player_i.loc[:,"MIN":].columns])
                              ],axis=1)
        
    df_player = pd.concat([player_i.loc[:,"GAME_ID":"TEAM_ID"].reset_index(drop=True),df_player],axis=1)
    df_ = pd.concat([df_, df_player],axis=0).reset_index(drop=True)

In [ ]:
len(df_)

In [ ]:
df_.to_pickle("/Users/chienan/Pycon/github/SportLottery/input/BOXSCOREADVANCE_PLAYER_LAST_10_GAEMS_2012to2019.pkl")

# Add Player Data to Team

In [ ]:
# aggregate data
df_ = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/BOXSCOREADVANCE_PLAYER_LAST_10_GAEMS_2012to2019.pkl")

In [ ]:
playerBoxScoreAdv = playerBoxScoreAdv.reset_index(drop=True)

In [ ]:
playerBoxScoreAdv = playerBoxScoreAdv.sort_values(["PLAYER_ID","GAME_ID"]).reset_index(drop=True)

In [ ]:
game_shift = playerBoxScoreAdv.groupby(["PLAYER_ID"])["GAME_ID"].shift().to_frame("GAME_ID_LAST")

In [ ]:
game_shift = pd.concat([game_shift,playerBoxScoreAdv[["GAME_ID","PLAYER_ID"]]],axis=1)

In [ ]:
df_["PLAYER_ID"] = df_.PLAYER_ID.astype(str)

In [ ]:
game_shift = game_shift.merge(df_, left_on=["GAME_ID_LAST","PLAYER_ID"], right_on=["GAME_ID","PLAYER_ID"],suffixes=["","_LAST"], how="left")

In [ ]:
game_shift.drop(["GAME_ID_LAST"],axis=1,inplace=True)

In [ ]:
game_shift = game_shift[~game_shift.TEAM_ID.isna()].reset_index(drop=True)

In [ ]:
game_shift = game_shift.fillna(0)

In [ ]:
game_shift["TEAM_ID"] = game_shift.TEAM_ID.astype(int)
game_shift["TEAM_ID"] = game_shift.TEAM_ID.astype(str)

In [ ]:
game_shift = game_shift.sort_values(["GAME_ID","TEAM_ID","PLAYER_ID"]).reset_index(drop=True)

In [ ]:
game_shift["PLAYER_MIN_RATIO"] = game_shift.groupby(["GAME_ID","TEAM_ID"]).apply(lambda x : x["LAST_1_GAME_PLAYER_MIN"]/sum(x["LAST_1_GAME_PLAYER_MIN"])).values

In [ ]:
game_shift = game_shift[~game_shift.PLAYER_MIN_RATIO.isna()].reset_index(drop=True)

In [ ]:
game_shift.loc[:,"LAST_1_GAME_PLAYER_MIN":] = game_shift.loc[:,"LAST_1_GAME_PLAYER_MIN":].apply(lambda x : (x * x["LAST_1_GAME_PLAYER_MIN"])/240.0,axis=1)

In [ ]:
game_shift = game_shift.groupby(["GAME_ID","TEAM_ID"]).sum().reset_index()

In [ ]:
len(game_shift)

# Combine Player and Team infomation

In [ ]:
nba_df = nba_df[~nba_df.LAST_10_GAME_PLUS_MINUS_A.isna()].reset_index(drop=True)

In [ ]:
len(nba_df)

In [ ]:
len(game_shift)

In [ ]:
nba_df["TEAM_ID_HOME"] = nba_df["TEAM_ID_HOME"].astype(str)
nba_df["TEAM_ID_AWAY"] = nba_df["TEAM_ID_AWAY"].astype(str)
game_shift["TEAM_ID"] = game_shift["TEAM_ID"].astype(str)

In [ ]:
game_shift = (nba_df[["GAME_ID", "TEAM_ID_HOME", "TEAM_ID_AWAY"]].merge(game_shift,left_on=["GAME_ID","TEAM_ID_HOME"],right_on=["GAME_ID","TEAM_ID"],how="left")
              .merge(game_shift,left_on=["GAME_ID","TEAM_ID_AWAY"],right_on=["GAME_ID","TEAM_ID"],how="left",suffixes=['_H','_A']))

In [ ]:
game_shift.drop(["TEAM_ID_H","TEAM_ID_A","PLAYER_MIN_RATIO_A","PLAYER_MIN_RATIO_H"],axis=1,inplace=True)

In [ ]:
gc.collect()

In [ ]:
nba_df = nba_df.merge(game_shift, on=["GAME_ID", "TEAM_ID_HOME", "TEAM_ID_AWAY"],how="inner" )
nba_df = nba_df.sort_values("GAME_ID").reset_index(drop=True)

In [ ]:
nba_df.tail()

In [ ]:
len(df)

## calculate distance 

In [4]:
def haversine(coord1, coord2):
    '''
    calculate the great circle distance between two points
    on the Earth (coordinates specified in decimal degrees)
    '''
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [coord1[0], coord1[1], coord2[0], coord2[1]])
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    # 6367 km (3961 miles) is the radius of the Earth
    km = 6367 * c
    miles = 3961 * c
    return km, miles

In [5]:
nba_df = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/FEATURE_AGG_2012TO2019.pkl").sort_values("GAME_ID").reset_index(drop=True)
team_coord = pickle.load(open("/Users/chienan/Pycon/github/SportLottery/input/city_dist.pkl","rb"))

In [6]:
game = nba_df.loc[:,:"TEAM_ID_AWAY"]
game["GAME_LOCATION"] = game.TEAM_ID_HOME.apply(lambda x : team_coord[x])

In [9]:
game_h = game.loc[:,["GAME_ID","TEAM_ID_HOME","GAME_LOCATION"]].rename({"TEAM_ID_HOME":"TEAM_ID"},axis=1)


In [10]:
game_a = game.loc[:,["GAME_ID","TEAM_ID_AWAY","GAME_LOCATION"]].rename({"TEAM_ID_AWAY":"TEAM_ID"},axis=1)


In [11]:
game = pd.concat([game.loc[:,["GAME_ID","TEAM_ID_HOME","GAME_LOCATION"]].rename({"TEAM_ID_HOME":"TEAM_ID"},axis=1)
                  ,game.loc[:,["GAME_ID","TEAM_ID_AWAY","GAME_LOCATION"]].rename({"TEAM_ID_AWAY":"TEAM_ID"},axis=1)],
                axis=0).reset_index(drop=True)

In [12]:
game = game.sort_values(["TEAM_ID","GAME_ID"]).reset_index(drop=True)
game["YEAR_ID"] = game.GAME_ID.apply(lambda x : x[:5]) 

In [14]:
game["LAST_GAME_LOCATION"] = game.groupby(["TEAM_ID","YEAR_ID"]).GAME_LOCATION.shift(1)

In [15]:
game.loc[game[game.LAST_GAME_LOCATION.isna()].index,"LAST_GAME_LOCATION"] = game.loc[game[game.LAST_GAME_LOCATION.isna()].index,"GAME_LOCATION"]

In [16]:
game["TRAVEL_DISTANCE"] = game.apply(lambda g : haversine(g["LAST_GAME_LOCATION"],g["GAME_LOCATION"])[1],axis=1)

In [20]:
game.head()

,GAME_ID,TEAM_ID,GAME_LOCATION,YEAR_ID,LAST_GAME_LOCATION,TRAVEL_DISTANCE
0,0021100274,1610612737,"(42.3315509, -83.0466403)",00211,"(42.3315509, -83.0466403)",0.000000
1,0021100296,1610612737,"(29.9499323, -90.0701156)",00211,"(42.3315509, -83.0466403)",485.438713
2,0021100308,1610612737,"(43.653963, -79.387207)",00211,"(29.9499323, -90.0701156)",738.398983
3,0021100326,1610612737,"(33.7490987, -84.3901849)",00211,"(43.653963, -79.387207)",357.849660
4,0021100362,1610612737,"(33.7490987, -84.3901849)",00211,"(33.7490987, -84.3901849)",0.000000


In [22]:
game.drop(["GAME_LOCATION","YEAR_ID","LAST_GAME_LOCATION"],axis=1,inplace=True)

In [18]:
nba_df = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/FEATURE_AGG_2012TO2019.pkl")

In [29]:
nba_df = nba_df.merge(game.rename({"TEAM_ID":"TEAM_ID_HOME"},axis=1),
                      how="inner",on=["GAME_ID","TEAM_ID_HOME"])
nba_df = nba_df.merge(game.rename({"TEAM_ID":"TEAM_ID_AWAY"},axis=1),
                      how="inner",on=["GAME_ID","TEAM_ID_AWAY"],
                      suffixes=["_H","_A"])

In [31]:
nba_df.head()

,GAME_ID,TEAM_ID_HOME,TEAM_ID_AWAY,PLUS_MINUS,LAST_1_GAME_FGM_H,LAST_1_GAME_FGA_H,LAST_1_GAME_FG_PCT_H,LAST_1_GAME_FG3M_H,LAST_1_GAME_FG3A_H,LAST_1_GAME_FG3_PCT_H,LAST_1_GAME_FTM_H,LAST_1_GAME_FTA_H,LAST_1_GAME_FT_PCT_H,LAST_1_GAME_OREB_H,LAST_1_GAME_DREB_H,LAST_1_GAME_REB_H,LAST_1_GAME_AST_H,LAST_1_GAME_STL_H,LAST_1_GAME_BLK_H,LAST_1_GAME_TOV_H,LAST_1_GAME_PF_H,LAST_1_GAME_PTS_H,LAST_1_GAME_WL_H,LAST_2_GAME_FGM_H,LAST_2_GAME_FGA_H,LAST_2_GAME_FG_PCT_H,LAST_2_GAME_FG3M_H,LAST_2_GAME_FG3A_H,LAST_2_GAME_FG3_PCT_H,LAST_2_GAME_FTM_H,LAST_2_GAME_FTA_H,LAST_2_GAME_FT_PCT_H,LAST_2_GAME_OREB_H,LAST_2_GAME_DREB_H,LAST_2_GAME_REB_H,LAST_2_GAME_AST_H,LAST_2_GAME_STL_H,LAST_2_GAME_BLK_H,LAST_2_GAME_TOV_H,LAST_2_GAME_PF_H,LAST_2_GAME_PTS_H,LAST_2_GAME_WL_H,LAST_3_GAME_FGM_H,LAST_3_GAME_FGA_H,LAST_3_GAME_FG_PCT_H,LAST_3_GAME_FG3M_H,LAST_3_GAME_FG3A_H,LAST_3_GAME_FG3_PCT_H,LAST_3_GAME_FTM_H,LAST_3_GAME_FTA_H,LAST_3_GAME_FT_PCT_H,LAST_3_GAME_OREB_H,LAST_3_GAME_DREB_H,LAST_3_GAME_REB_H,LAST_3_GAME_AST_H,LAST_3_GAME_STL_H,LAST_3_GAME_BLK_H,LAST_3_GAME_TOV_H,LAST_3_GAME_PF_H,LAST_3_GAME_PTS_H,LAST_3_GAME_WL_H,LAST_4_GAME_FGM_H,LAST_4_GAME_FGA_H,LAST_4_GAME_FG_PCT_H,LAST_4_GAME_FG3M_H,LAST_4_GAME_FG3A_H,LAST_4_GAME_FG3_PCT_H,LAST_4_GAME_FTM_H,LAST_4_GAME_FTA_H,LAST_4_GAME_FT_PCT_H,LAST_4_GAME_OREB_H,LAST_4_GAME_DREB_H,LAST_4_GAME_REB_H,LAST_4_GAME_AST_H,LAST_4_GAME_STL_H,LAST_4_GAME_BLK_H,LAST_4_GAME_TOV_H,LAST_4_GAME_PF_H,LAST_4_GAME_PTS_H,LAST_4_GAME_WL_H,LAST_5_GAME_FGM_H,LAST_5_GAME_FGA_H,LAST_5_GAME_FG_PCT_H,LAST_5_GAME_FG3M_H,LAST_5_GAME_FG3A_H,LAST_5_GAME_FG3_PCT_H,LAST_5_GAME_FTM_H,LAST_5_GAME_FTA_H,LAST_5_GAME_FT_PCT_H,LAST_5_GAME_OREB_H,LAST_5_GAME_DREB_H,LAST_5_GAME_REB_H,LAST_5_GAME_AST_H,LAST_5_GAME_STL_H,LAST_5_GAME_BLK_H,LAST_5_GAME_TOV_H,LAST_5_GAME_PF_H,LAST_5_GAME_PTS_H,LAST_5_GAME_WL_H,LAST_6_GAME_FGM_H,LAST_6_GAME_FGA_H,LAST_6_GAME_FG_PCT_H,LAST_6_GAME_FG3M_H,LAST_6_GAME_FG3A_H,LAST_6_GAME_FG3_PCT_H,LAST_6_GAME_FTM_H,LAST_6_GAME_FTA_H,LAST_6_GAME_FT_PCT_H,LAST_6_GAME_OREB_H,LAST_6_GAME_DREB_H,LAST_6_GAME_REB_H,LAST_6_GAME_AST_H,LAST_6_GAME_STL_H,LAST_6_GAME_BLK_H,LAST_6_GAME_TOV_H,LAST_6_GAME_PF_H,LAST_6_GAME_PTS_H,LAST_6_GAME_WL_H,LAST_7_GAME_FGM_H,LAST_7_GAME_FGA_H,LAST_7_GAME_FG_PCT_H,LAST_7_GAME_FG3M_H,LAST_7_GAME_FG3A_H,LAST_7_GAME_FG3_PCT_H,LAST_7_GAME_FTM_H,LAST_7_GAME_FTA_H,LAST_7_GAME_FT_PCT_H,LAST_7_GAME_OREB_H,LAST_7_GAME_DREB_H,LAST_7_GAME_REB_H,LAST_7_GAME_AST_H,LAST_7_GAME_STL_H,LAST_7_GAME_BLK_H,LAST_7_GAME_TOV_H,LAST_7_GAME_PF_H,LAST_7_GAME_PTS_H,LAST_7_GAME_WL_H,LAST_8_GAME_FGM_H,LAST_8_GAME_FGA_H,LAST_8_GAME_FG_PCT_H,LAST_8_GAME_FG3M_H,LAST_8_GAME_FG3A_H,LAST_8_GAME_FG3_PCT_H,LAST_8_GAME_FTM_H,LAST_8_GAME_FTA_H,LAST_8_GAME_FT_PCT_H,LAST_8_GAME_OREB_H,LAST_8_GAME_DREB_H,LAST_8_GAME_REB_H,LAST_8_GAME_AST_H,LAST_8_GAME_STL_H,LAST_8_GAME_BLK_H,LAST_8_GAME_TOV_H,LAST_8_GAME_PF_H,LAST_8_GAME_PTS_H,LAST_8_GAME_WL_H,LAST_9_GAME_FGM_H,LAST_9_GAME_FGA_H,LAST_9_GAME_FG_PCT_H,LAST_9_GAME_FG3M_H,LAST_9_GAME_FG3A_H,LAST_9_GAME_FG3_PCT_H,LAST_9_GAME_FTM_H,LAST_9_GAME_FTA_H,LAST_9_GAME_FT_PCT_H,LAST_9_GAME_OREB_H,LAST_9_GAME_DREB_H,LAST_9_GAME_REB_H,LAST_9_GAME_AST_H,LAST_9_GAME_STL_H,LAST_9_GAME_BLK_H,LAST_9_GAME_TOV_H,LAST_9_GAME_PF_H,LAST_9_GAME_PTS_H,LAST_9_GAME_WL_H,LAST_10_GAME_FGM_H,LAST_10_GAME_FGA_H,LAST_10_GAME_FG_PCT_H,LAST_10_GAME_FG3M_H,LAST_10_GAME_FG3A_H,LAST_10_GAME_FG3_PCT_H,LAST_10_GAME_FTM_H,LAST_10_GAME_FTA_H,LAST_10_GAME_FT_PCT_H,LAST_10_GAME_OREB_H,LAST_10_GAME_DREB_H,LAST_10_GAME_REB_H,LAST_10_GAME_AST_H,LAST_10_GAME_STL_H,LAST_10_GAME_BLK_H,LAST_10_GAME_TOV_H,LAST_10_GAME_PF_H,LAST_10_GAME_PTS_H,LAST_10_GAME_WL_H,LAST_1_GAME_FGM_A,LAST_1_GAME_FGA_A,LAST_1_GAME_FG_PCT_A,LAST_1_GAME_FG3M_A,LAST_1_GAME_FG3A_A,LAST_1_GAME_FG3_PCT_A,LAST_1_GAME_FTM_A,LAST_1_GAME_FTA_A,LAST_1_GAME_FT_PCT_A,LAST_1_GAME_OREB_A,LAST_1_GAME_DREB_A,LAST_1_GAME_REB_A,LAST_1_GAME_AST_A,LAST_1_GAME_STL_A,LAST_1_GAME_BLK_A,LAST_1_GAME_TOV_A,LAST_1_GAME_PF_A,LAST_1_GAME_PTS_A,LAST_1_GAME_WL_A,LAST_2_GAME_FGM_A,LAST_2_GAME_FGA_A,LAST_

In [32]:
nba_df.to_pickle("/Users/chienan/Pycon/github/SportLottery/input/FEATURE_AGG_2012TO2019.pkl")

# Lottery Data

In [ ]:
lottery_df = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LOTTERY_2012to2019.pkl")

In [ ]:
team_coord

In [ ]:
team = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_HOME_2012to2019.pkl")[['GAME_ID', 'TEAM_ID', 'GAME_DATE']].reset_index(drop=True)
team = team.merge(pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_AWAY_2012to2019.pkl")[['GAME_ID', 'TEAM_ID', 'GAME_DATE']].reset_index(drop=True),
                 on=["GAME_ID","GAME_DATE"],how="inner",suffixes=["_HOME","_AWAY"])

In [ ]:
team["GAME_DATE"] = team.GAME_DATE.apply(lambda dt : datetime.strptime(dt,"%Y-%m-%d").date())

In [ ]:
lottery_with_gameid = lottery_df.merge(team, on=["TEAM_ID_HOME","TEAM_ID_AWAY","GAME_DATE"], how="inner")

In [ ]:
lottery_df["GAME_DATE"] = lottery_df.GAME_DATE.apply(lambda dt : dt + timedelta(days=-1))
lottery_with_gameid = pd.concat([lottery_with_gameid,lottery_df.merge(team, on=["TEAM_ID_HOME","TEAM_ID_AWAY","GAME_DATE"], how="inner")],
                                axis=0).reset_index(drop=True)

In [ ]:
lottery_with_gameid = lottery_with_gameid.loc[:,[col for col in lottery_with_gameid.columns if col not in 
                                                 ['AWAY','HOME','TEAM_CH_NAME_HOME', 'TEAM_EN_NAME_HOME', 
                                                  'TEAM_CH_NAME_AWAY', 'TEAM_EN_NAME_AWAY','GAME_DATE']]]

In [ ]:
lottery_with_gameid.to_pickle("GAMEID_LOTTERY_2012to2019.pkl")

In [ ]:
lottery_df